# SQL Database for Blockbuster

*Commit 1*

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

import mysql.connector as conn

## 1. Create new database called blockbuster and connect to it

In [2]:
conexion = conn.connect(host='localhost',
                        user='root',
                        passwd='password'
                       )

cursor = conexion.cursor()


cursor

In [3]:
cursor.execute('drop database if exists blockbuster;')

cursor.execute('create database blockbuster;')

conexion = conn.connect(host='localhost',
                        user='root',
                        passwd='password',
                        database='blockbuster'
                       )


cursor = conexion.cursor()

c=cursor.execute

c


<bound method MySQLCursor.execute of <mysql.connector.cursor.MySQLCursor object at 0x7f8a3effb9d0>>

## 2. Look at the Data

In [4]:
actor = pd.read_csv('../data/actor.csv')
categ = pd.read_csv('../data/category.csv')
film = pd.read_csv('../data/film.csv')
invent = pd.read_csv('../data/inventory.csv')
lang = pd.read_csv('../data/language.csv')
old = pd.read_csv('../data/old_HDD.csv')
rental = pd.read_csv('../data/rental.csv')

*Commit 2,3 y 4*

## 3. Data Cleaning

### Deleting update time from all the columns

In [12]:
x='last_update'

actor.drop(x, axis=1, inplace= True)
lang.drop(x, axis=1, inplace= True)
rental.drop(x, axis=1, inplace= True)
invent.drop(x, axis=1, inplace= True)
film.drop(x, axis=1, inplace= True)
categ.drop(x, axis=1, inplace= True)

### Invent/Films fix

In [13]:
film.drop('original_language_id', axis=1, inplace= True)

### Deleting duplicated actor

In [14]:
actor.drop(actor[actor.duplicated(subset=['first_name', 'last_name'])].index, inplace=True)

### Changing the actor names from 'old' to actor_id

In [17]:
old['actor_id'] = pd.Series([np.nan] * len(old))
old['full_name'] = old['first_name'] + ' ' + old['last_name']

del old['first_name']
del old['last_name']



In [18]:
actor['full_name'] = actor['first_name'] + ' ' + actor['last_name']

del actor['first_name']
del actor['last_name']


*commit 5*

In [25]:
actores_ides = dict(zip(actor.full_name.values,actor.actor_id.values))


In [22]:
old['actor_id'] = [actores_ides[i] for i in old.full_name]

In [35]:
film_ides = dict(zip(film.title.values,film.film_id.values))

In [29]:
old['title_id'] = [film_ides[i] for i in old.title]

In [33]:
bad_columns= ['release_year', 'full_name']

old.drop(columns=bad_columns, inplace=True)

old

,title,category_id,actor_id,title_id
0,ACADEMY DINOSAUR,6,1,1
1,ANACONDA CONFESSIONS,2,1,23
2,ANGELS LIFE,13,1,25
3,BULWORTH COMMANDMENTS,10,1,106
4,CHEAPER CLYDE,14,1,140
...,...,...,...,...
995,COMANCHEROS ENEMY,3,39,168
996,DAISY MENAGERIE,14,39,203
997,DESERT POSEIDON,11,39,222
998,EVERYONE CRAFT,9,39,290


*Commit 6*

### Category  to film table